# Ramp 28 -- CNN

In [1]:
#Data processing
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from nptdms import TdmsFile #Process ramping file

#For building ML models
import keras
import keras.models as models
from keras.layers.core import Dense
from keras import regularizers
from keras.layers.convolutional import Conv1D, MaxPooling1D, AveragePooling1D
from keras.layers import UpSampling1D
from keras.layers.core import Reshape,Dense,Dropout,Activation,Flatten


Bad key "text.kerning_factor" on line 4 in
/uscms_data/d3/dhoang/miniconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution
Using TensorFlow backend.


## 0. Helper functions

In [2]:
def load_channel_and_time(dir_path, channel):
    data_frame = pd.DataFrame(data = {channel: np.load(dir_path + channel + ".npy"),
                                     "time": np.load(dir_path + "time.npy")})
    return data_frame

In [3]:
from sklearn.preprocessing import minmax_scale

def rolling_window(data, window_size, step):
    """Calculate rolling window values over a (numpy) time series data,
    and return a matrix of shape (# data points, window_size).
    Note: The values in each window is scaled to between (0,1)
    
    Example: rolling_window(data["ai0"].to_numpy(), window_size = 2000, step = 300)"""
    
    X = []
    
    #Loop over data to collect window
    for i in range(0, data.shape[0]-window_size, step):
        #Select window
        X.append(minmax_scale(data[i:i+window_size]))
        
    X = np.asarray(X)
    
    %reset -f in
    
    return X

def _get_window_and_time(dir_path, sensor, time_range, window_size = 2000, step = 10):
    """
    Load a single sensor's data and calculate the standard deviations of
    rolling windows.
    
    Return: Standard deviation calculations, time
    """
    
    #Load the data
    data = load_channel_and_time(dir_path, sensor)
    
    #Select the part
    start = time_range[0]
    end = time_range[1]
    data = data[(data["time"] > start) & (data["time"] < end)]
    
    X = rolling_window(data[sensor].to_numpy(), window_size=window_size, step=step)
    
    #Make sure the time line up
    assert X.shape[0] == data['time'].to_numpy()[window_size-1::step].shape[0]
    
    %reset -f in
    
    return X , data['time'].to_numpy()[window_size-1::step]

## 1. Obtain data & Visualization

In [4]:
X_train, train_time = _get_window_and_time("./data/Ramp28/", "ai2", time_range = (-300, - 200), window_size = 2000, step = 100)
X_test, test_time = _get_window_and_time("./data/Ramp28/", "ai2", time_range = (-50, 0), window_size = 2000, step = 100)

Flushing input history
Flushing input history
Flushing input history
Flushing input history


In [5]:
X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)

## 2. Build model

In [6]:
#================BUILD THE MODEL====================
input_shape = X_train.shape[1:]
print("Input shape: ", input_shape)
print("Using Keras version: ", keras.__version__)

#Conv1d model
model = models.Sequential()

#Encoder
model.add(Conv1D(3, 100, activation='elu', padding = "same", kernel_initializer='glorot_uniform', input_shape=(2000, 1)))
model.add(AveragePooling1D(2))

model.add(Conv1D(3, 100, activation='elu', padding = "same", kernel_initializer='glorot_uniform', input_shape=(2000, 1)))
model.add(MaxPooling1D(2))

model.add(Conv1D(3, 100, activation='elu', padding = "same", kernel_initializer='glorot_uniform', input_shape=(2000, 1)))
model.add(MaxPooling1D(2))

model.add(Flatten())

#Hidden feature descriptor
model.add(Dense(300, activation='elu', kernel_initializer='he_normal'))

model.add(Dense(750, activation='elu', kernel_initializer='he_normal'))
model.add(Reshape((250, 3)))

#Decoder
model.add(Conv1D(3, 100, activation='elu', padding = "same", kernel_initializer='glorot_uniform'))
model.add(UpSampling1D(2))

model.add(Conv1D(3, 100, activation='elu', padding = "same", kernel_initializer='glorot_uniform'))
model.add(UpSampling1D(2))

model.add(Conv1D(3, 100, activation='elu', padding = "same", kernel_initializer='glorot_uniform'))
model.add(UpSampling1D(2))


#Recontruct input
model.add(Conv1D(1, 100, activation='elu', padding = "same", kernel_initializer='glorot_uniform'))

model.compile(loss='mse', optimizer="adam")
model.summary()

Input shape:  (2000, 1)
Using Keras version:  2.3.1
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 2000, 3)           303       
_________________________________________________________________
average_pooling1d_1 (Average (None, 1000, 3)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 1000, 3)           903       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 500, 3)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 500, 3)            903       
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 250, 3)            0         
_________________________________________________________________
fl

In [7]:
# Train model for 100 epochs, batch size of 10: 
NUM_EPOCHS=50
BATCH_SIZE=1024

history=model.fit(X_train, X_train,
                  batch_size=BATCH_SIZE, 
                  epochs=NUM_EPOCHS,
                  validation_split=0.1,
                  verbose = 1)

Train on 89982 samples, validate on 9998 samples
Epoch 1/50
89982/89982 [==============================] - 354s 4ms/step - loss: 0.0217 - val_loss: 0.0084
Epoch 2/50
89982/89982 [==============================] - 325s 4ms/step - loss: 0.0073 - val_loss: 0.0077
Epoch 3/50
89982/89982 [==============================] - 327s 4ms/step - loss: 0.0070 - val_loss: 0.0075
Epoch 4/50
89982/89982 [==============================] - 340s 4ms/step - loss: 0.0071 - val_loss: 0.0074
Epoch 5/50
89982/89982 [==============================] - 342s 4ms/step - loss: 0.0072 - val_loss: 0.0075
Epoch 6/50
62464/89982 [===================>..........] - ETA: 1:42 - loss: 0.0068

KeyboardInterrupt: 

In [9]:
X_pred = model.predict(np.array(X_train))
scored = pd.DataFrame()
scored['Loss_mse'] = np.mean(np.abs(X_pred-X_train), axis = 1)

MemoryError: Unable to allocate 1.49 GiB for an array with shape (99980, 2000, 1) and data type float64

In [ ]:
X_pred_test = model.predict(np.array(X_test))
scored_test = pd.DataFrame()
scored_test['Loss_mse'] = np.mean(np.abs(X_pred_test-X_test), axis = 1)

In [ ]:
plt.figure(figsize=(20,2))
plt.plot(train_time, scored['Loss_mse'])
plt.ylim([0, 0.5])
plt.xlabel("Time (s)")

In [ ]:
plt.figure(figsize=(20,2))
plt.plot(test_time, scored_test['Loss_mse'])
plt.ylim([0,0.5])
#plt.xlim([-1,0])
plt.xlabel("Time (s)")